In [ ]:
import torch
from model.LightGCN import *
from preprocess.Gowalla import *
from evaluation.LightGCN_SISA import *
pd.options.display.max_rows = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
path = './dataset/gowalla'
dataset = Gowalla(path)

# Data(num_nodes=144242, edge_index=[2, 2380730], edge_label_index=[2, 603378])
data = dataset.get()
num_users, num_books = dataset.getNumber()
config = {
    'k': 20,
    'lr': 0.001,
    'epochs': 1000,
    'num_layers': 2,
    'batch_size': 8192,
    'embedding_dim': 64,
    'num_users': num_users,
    'num_books': num_books,
    'tuning_type': None,
}
model = LightGCN(
    num_nodes=data.num_nodes,
    embedding_dim=config['embedding_dim'],
    num_layers=config['num_layers'],
).to(device)

In [ ]:
config['epochs'] = 500
config['num_shards'] = 5
model_list =[]

for _ in range(config['num_shards']):
    model_list.append(LightGCN(
    num_nodes=data.num_nodes,
    embedding_dim=config['embedding_dim'],
    num_layers=config['num_layers'],
).to(device))

shard_models, shards, epoch_tracks, test_topks = sisa_lightgcn_eva(model_list, config, data, device)

In [ ]:
# Basic setting
num_edges = data.edge_index.size(1)
perm = torch.randperm(num_edges)
split = int(num_edges * 0.1)  # 10% of edges will be retained

# Define the data for forget and retain dataset
forget_data = Data()
retain_data = Data()

forget_data.num_nodes = data.num_nodes
retain_data.num_nodes = data.num_nodes
forget_data.edge_index = data.edge_index[:, perm[:split]]
retain_data.edge_index = data.edge_index[:, perm[split:]]
forget_data.edge_label_index = data.edge_label_index
retain_data.edge_label_index = data.edge_label_index

In [ ]:
shard_models = sisa_lightgcn_unlearning_eva(shard_models, shards, retain_data, forget_data, config, device)

In [ ]:
sisa_lightgcn_forget_data_eva(shard_models, forget_data, config, device)